Import Moule

In [2]:
import os
import numpy as np
import librosa
import joblib
import matplotlib.pyplot as plt
import tensorflow as tf 


Load Models

In [63]:
from tensorflow.keras.models import load_model
G_NAM2WHSP= load_model(r"C:\Users\SID\Documents\Speech\voice_conversion_gan\models\G_NAM2WHSP.keras")

In [64]:
G_NAM2WHSP.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 512)            │    20,667,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu (ReLU)                    │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_1 (ReLU)                  │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_2 (ReLU)                  │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 40365)          │    20,707,245 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 41,899,949 (159.84 MB)

 Trainable params: 41,899,949 (159.84 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
 
# Method to do padding 
def pad_audio(y, max_samples=1589449):
    padding_length = max_samples - len(y)
    if padding_length > 0:
        y = np.pad(y, (0, padding_length), 'constant')
    return y

# 'librosa.load'  loads the audio file, returning the audio time series (y) and the sampling rate (sr)
signal,sr=librosa.load(r"C:\Users\SID\Documents\Speech\voice_conversion_gan\data\raw\001_nam.wav", sr=None)
signal=pad_audio(signal)
# Computes the MFCCs of the audio signal. MFCCs are a representation of the short-term power spectrum of sound.
mfccs = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=13).flatten()

from tensorflow.keras.models import load_model
G_NAM2WHSP= load_model(r"C:\Users\SID\Documents\Speech\voice_conversion_gan\models\G_NAM2WHSP.keras")
test_nam_mfcc = tf.convert_to_tensor(mfccs, dtype=tf.float32)
nam = tf.reshape(test_nam_mfcc, (1, -1))
pred_mfccs=G_NAM2WHSP.predict(nam)
# mfcc_original_shape = mfcc_flattened.reshape(original_shape)  # Reshape to original shape
mfcc_original_shape = pred_mfccs.reshape((13,3105))
mel_spectrogram = librosa.feature.inverse.mfcc_to_mel(mfcc_original_shape)
stft = librosa.feature.inverse.mel_to_stft(mel_spectrogram)
audio_reconstructed = librosa.istft(stft)


In [47]:
nam_mfcc_features= joblib.load(r"C:\Users\SID\Documents\Speech\voice_conversion_gan\data\processed\mfcc_nam.pkl")
whsp_mfcc_features=joblib.load(r"C:\Users\SID\Documents\Speech\voice_conversion_gan\data\processed\mfcc_headset.pkl")

In [59]:

test_nam_mfcc = tf.convert_to_tensor(nam_mfcc_features[21], dtype=tf.float32)
nam = tf.reshape(test_nam_mfcc, (1, -1))
mfccs=G_NAM2WHSP.predict(nam)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step


In [60]:
# mfcc_original_shape = mfcc_flattened.reshape(original_shape)  # Reshape to original shape
mfcc_original_shape = mfccs.reshape((13,3105))
mel_spectrogram = librosa.feature.inverse.mfcc_to_mel(mfcc_original_shape)
stft = librosa.feature.inverse.mel_to_stft(mel_spectrogram)
audio_reconstructed = librosa.istft(stft)

In [62]:
import soundfile as sf
sf.write("test_gen3.wav", audio_reconstructed, 96000)

In [53]:
whsp_mfcc_features[21]

array([-646.5383 , -653.8847 , -668.99994, ...,    0.     ,    0.     ,
          0.     ], dtype=float32)

In [27]:
test_nam_path=r"C:\Users\SID\Documents\Speech\voice_conversion_gan\data\raw\001_nam.wav"
test_nam_signal,test_nam_sr =librosa.load(test_nam_path, sr=None)
test_nam_mfcc=librosa.feature.mfcc(y=test_nam_signal, sr=test_nam_sr, n_mfcc=13)

test_nam_mfcc = tf.convert_to_tensor(test_nam_mfcc, dtype=tf.float32)
 # Reshape data for inference
nam = tf.reshape(test_nam_mfcc, (1, -1))

In [28]:
G_NAM2WHSP.predict(nam)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 861ms/step


array([[-802.7878   ,  174.34099  ,   -3.6729894,   42.86202  ,
           3.5532148,   21.598917 ,  -10.459182 ,   32.130955 ,
          12.2100725,   13.005962 ,    9.805557 ,   13.93644  ,
          10.381048 ]], dtype=float32)

In [25]:
print(G_NAM2WHSP.input_shape)

(None, 13)


In [26]:
print(test_nam_mfcc.shape)

(13,)


In [31]:
mfcc_features = np.reshape(G_NAM2WHSP.predict(nam), (1, -1, 13))
# Compute the inverse MFCC using librosa
S = librosa.feature.inverse.mfcc_to_mel(mfcc_features.T)
# Reconstruct the audio signal from the Mel spectrogram
audio = librosa.feature.inverse.mel_to_audio(S, sr=96000, n_fft=13, hop_length=512)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step


c:\Users\SID\anaconda3\Lib\site-packages\librosa\feature\inverse.py:99: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = filters.mel(
c:\Users\SID\anaconda3\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=13 is too large for input signal of length=1
  warnings.warn(


In [32]:
audio

array([[ 9.4173020e-03],
       [-1.9600459e+02],
       [ 3.2029354e+01],
       [ 5.1428734e+01],
       [ 3.4473373e+01],
       [ 4.1422401e+01],
       [-2.9892149e+01],
       [ 4.6108475e+01],
       [ 3.7648014e+01],
       [ 3.7954166e+01],
       [-3.6737999e+01],
       [ 3.8315243e+01],
       [ 3.6953777e+01]], dtype=float32)

In [35]:
import soundfile as sf
sf.write("test_gen.wav", audio, 96000)